#Random Forest


## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.

In [2]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.metrics import *
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader
from math import sqrt


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/01.Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data('non-secuencial')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Mostrando los datos de data_total_for_non_serial.csv
(30555, 34)
________________________________________________________________________________


## Preparación de los datos

In [3]:
x_data = esios_controller.get_data_real_time()
y_data = esios_controller.get_target_data()


# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (20471, 30)
Ytrain_dim: (20471, 1)


## Modelo

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regressor = RandomForestRegressor(random_state=0, n_estimators=100)
regressor.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [6]:
print('MSE en entrenamiento es: ', mean_squared_error(regressor.predict(x_train), y_train))
print('MSE en validación es: ', mean_squared_error(regressor.predict(x_valid), y_valid))

x_predict = regressor.predict(x_valid)
print('MAE: ', mean_absolute_error(x_predict, y_valid))
print('MedianAE: ', median_absolute_error(x_predict, y_valid))
print('RMSE: ', sqrt(mean_squared_error(x_predict, y_valid)))
print('log_MSE: ', mean_squared_log_error(x_predict, y_valid))
print('Variance: ', explained_variance_score(x_predict, y_valid))
print('Max Error: ', max_error(x_predict, y_valid))

MSE en entrenamiento es:  6.998705476154555
MSE en validación es:  91.83243530762302
MAE:  6.249774781832609
MedianAE:  4.776250000000047
RMSE:  9.582924152241999
RMSE:  0.0575167702931859
Variance:  -0.3422264327119322
Max Error:  116.1001


## Normalizados

In [0]:
scaler = StandardScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)

In [6]:
regressor_standarized = RandomForestRegressor(random_state=0, n_estimators=100)
regressor_standarized.fit(x_train_est, y_train_est)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [1]:
print('MSE en entrenamiento es: ', mean_squared_error(regressor.predict(x_train), y_train))
print('MSE en validación es: ', mean_squared_error(regressor.predict(x_valid), y_valid))

x_predict = regressor.predict(x_valid)
print('MAE: ', smean_absolute_error(x_predict, y_valid))
print('MAE: ', mean_absolute_error(x_predict, y_valid))
print('MedianAE: ', median_absolute_error(x_predict, y_valid))
print('log MSE: ', mean_squared_log_error(x_predict, y_valid))
print('Variance: ', explained_variance_score(x_predict, y_valid))
print('Max Error: ', max_error(x_predict, y_valid))

NameError: ignored

## Optimización de modelos

In [0]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [10, 100, 150, 200, 300, 500, 600, 750, 1000]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

#param_grid = {'n_estimators': n_estimators}
#param_grid = {'max_features': max_features}
#param_grid = {'max_depth': max_depth}
#param_grid = {'min_samples_split': min_samples_split}
#param_grid = {'min_samples_leaf': min_samples_leaf}
param_grid = {'bootstrap': bootstrap, 
             'min_samples_leaf': min_samples_leaf}

clf = GridSearchCV(regressor, param_grid, cv=2)
clf.fit(x_train, y_train)

In [0]:
 print(clf.best_params_)

* {'n_estimators': 1000}
* {'max_features': 'auto'}
* {'max_depth': 20}
* {'min_samples_split': 2}
* {'min_samples_leaf': 4}
* {'bootstrap': True}



In [0]:
regressor_optimun = RandomForestRegressor(
    random_state=0,
    n_estimators=300,
    max_features='auto',
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=4,
    bootstrap=True
)
regressor_optimun.fit(x_train, y_train)


print('MSE en entrenamiento es: ', mean_squared_error(regressor.predict(x_train), y_train))
print('MSE en validación es: ', mean_squared_error(regressor.predict(x_valid), y_valid))

x_predict = regressor.predict(x_valid)
print('MAE: ', mean_absolute_error(x_predict, y_valid))
print('MedianAE: ', median_absolute_error(x_predict, y_valid))
print('RMSE: ', mean_squared_log_error(x_predict, y_valid))
print('Variance: ', explained_variance_score(x_predict, y_valid))
print('Max Error: ', max_error(x_predict, y_valid))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


MSE en entrenamiento es:  6.998705476154555
MSE en validación es:  91.83243530762302
MAE:  6.249774781832609
MedianAE:  4.776250000000047
RMSE:  0.0575167702931859
Variance:  -0.3422264327119322
Max Error:  116.1001


In [8]:
regressor_optimun = RandomForestRegressor(
    random_state=0,
    n_estimators=1000,
    max_features='auto',
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=4,
    bootstrap=True
)
regressor_optimun.fit(x_train_est, y_train_est)


print('MSE en entrenamiento es: ', mean_squared_error(regressor.predict(x_train), y_train))
print('MSE en validación es: ', mean_squared_error(regressor.predict(x_valid), y_valid))

x_predict = regressor.predict(x_valid)
print('MAE: ', mean_absolute_error(x_predict, y_valid))
print('MedianAE: ', median_absolute_error(x_predict, y_valid))
print('RMSE: ', mean_squared_log_error(x_predict, y_valid))
print('Variance: ', explained_variance_score(x_predict, y_valid))
print('Max Error: ', max_error(x_predict, y_valid))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


MSE en entrenamiento es:  6.998705476154555
MSE en validación es:  91.83243530762302
MAE:  6.249774781832609
MedianAE:  4.776250000000047
RMSE:  0.0575167702931859
Variance:  -0.3422264327119322
Max Error:  116.1001
